**Sentence Parsing**

The goal of this script is to experiment with sentence parsing, to tightly associate nouns with their descriptive terminology

In [38]:
#load libraries, initialize spaCy

import os, glob
import numpy as np
import spacy as sp
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from spacy import displacy
import joblib
from multiprocessing import Pool
import time

#The 'xx' dataset is the biggest multilanguage one.  It catches the most names
#The 'en' dataset does the best job of parsing organizations and labels verbs and other parts of speech
# Install model with `python -m spacy download en`
nlp = sp.load('en')

In [54]:
# Load the genderize data
genderize = pd.read_csv("../source_data/genderizer_collected.csv").set_index('name')
genderize.head()
known_names = set(genderize.index.values)

In [9]:
# Sample read
sample = '<ack><title>Acknowledgements</title><p>The authors wish to acknowledge Diya Ma, Matthew-Lun Wong, Ka-Long Ko, Ka-Hei Ko and Jin-Peng Lee for their important contributions to the software development.</p><sec id=""FPar1""><title>Funding</title><p id=""Par28"">The work described in this paper was supported by a grant from the Research Grants Council of the Hong Kong Special Administrative Region, China (Project No.: CUHK 14113214), grants from the Innovation and Technology Commission (Project No: ITS/149/14FP, GHP/028/14SZ, ITS/293/14FP), grants from CUHK Technology and Business Development Fund (Project No.: TBF16MED002, TBF16MED004), a grant from The Science, Technology and Innovation Commission of Shenzhen Municipality (Project No.: CXZZ20140606164105361), and a grant from The Scientific Research Project of Guangdong Province (Project No.: 2014B090901055).</p></sec></ack>'
sample = '<ack id=""ack0010""><title>Acknowledgements</title><p>The authors thank Dr. R Kaneko for the gift of the iSip2 vector; and Mss. T Honma, K Harada, A Morita, and Y Shimoda for providing technical and secretarial assistance. We thank the staff at the Department of Genetic and Behavioral Neuroscience and Bioresource Center, Gunma University Graduate School of Medicine for their critical comments and technical assistance. This study was supported by <funding-source id=""gs1"">Grants-in-Aid for Scientific Research</funding-source> (23115503, 26290002, 15H01415 and 15H05872 to Y.Y.), a Grant-in-Aid for Scientific Research on Innovative Areas (Comprehensive Brain Science Network) (to Y.Y.) from the <funding-source id=""gs2"">Ministry of Education, Culture, Sports, Science and Technology (MEXT)</funding-source> of Japan, a grant from the Co-operative Study Program of the <funding-source id=""gs3"">National Institute for Physiological Sciences</funding-source>, Japan (to Y.Y.), and a grant from the <funding-source id=""gs4"">Takeda Science Foundation</funding-source> (to Y.Y.).</p></ack>'
sample = u'<ack id=""ack0005""><title>Acknowledgments</title><p>The project was supported by a start-up funding provided to the author by the <funding-source id=""gs0005"">Department of Neurology of the University of Utah</funding-source>.</p><p>This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.</p><p>I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.</p></ack>'

soup = BeautifulSoup(sample,'lxml')
for ele in soup.find_all('title'):
    ele.decompose()
samp_txt = soup.find_all('ack')[0].get_text(separator=' ')

print(samp_txt)
print()
doc = nlp(samp_txt)

for sent in doc.sents:
    if 'fund' in sent.text or 'grant' in sent.text: continue #if it's a funding sentence, we don't care
    print(sent.text)

The project was supported by a start-up funding provided to the author by the  Department of Neurology of the University of Utah . This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him. I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.

This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.
I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.


That's a good start, but we need to split up the individual parts of sentences which refer to multiple people

In [28]:
for sent in doc.sents:
    if 'fund' in sent.text or 'grant' in sent.text or 'supported' in sent.text or 'financ' in sent.text: continue #if it's a funding sentence, we don't care
    #count nouns
    sentdoc = nlp(sent.text) #if this needs speed optimization, we can search by start-stop characters instead
    sentppl = [n for n in sentdoc.ents if n.label_ == 'PERSON']
    people_count = len(sentppl)
    if people_count == 0: continue #no named people, disregard
    print(sent.text)

This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.
I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.


In [31]:
#actually run the thing on the real stuff
F_CSV = glob.glob("../source_data/extracted/*.csv")


In [126]:
def parse_df(row, k):
    if k%1000==0:
        print(k)
        if k%1000==0:
            print(time.asctime())
    
    if row.Acknowledgment_Tag is None or type(row.Acknowledgment_Tag) == float:
        return []
    
    soup = BeautifulSoup(row.Acknowledgment_Tag,'lxml')

    for ele in soup.find_all('title'):
        ele.decompose()
        
    text = soup.find('ack')
    if text is None:
        return []
    
    text = text.get_text(separator=' ')
    
    doc = nlp(text)
    sentlist=[]
    for sent in doc.sents:
        #check for funding terminology and skip the sentence if found
        fundwords = ['fund','grant','supported','financ','award']
        if any([f in sent.text for f in fundwords]): continue
        
        #count nouns
        badnamewords=['University','Research','Number','Doctoral','Postdoc','Scientist','Data','Surgeon','Centre','Center','Foundation','Universidad','Microscop','Microbio']
        sentdoc = nlp(sent.text) #if this needs speed optimization, we can search by start-stop characters instead
        sentppl = [n for n in sentdoc.ents if n.label_ == 'PERSON' and len(n.text.replace("\n"," ").strip().split(' ')) > 1 and not any(c.isdigit() for c in n.text) and not any([f in n.text for f in badnamewords])]
        people_count = len(sentppl)
        if people_count == 0: continue #no named people, disregard
            
        
            
        #at least one person, populate a new item and add it to the list
        item = {"filename":row.filename}
        item["Text"] = sent.text
        item["Verbs"] = ';'.join([ word.lemma_ for word in sentdoc if word.pos_ == 'VERB' and not word.is_stop])  
        item["Nouns"] = ';'.join([ word.text for word in sentdoc if word.pos_ == 'NOUN' and not word.is_stop])
        item["Names"] = ';'.join([ ent.text for ent in sentdoc.ents if ent.label_ == 'PERSON' and len(ent.text.split(' ')) > 1 ])
        item["Organizations"] = ';'.join([ ent.text for ent in sentdoc.ents if ent.label_ == 'ORG' ])
        
        genders=[]
        genderprobs=[]
        for name in sentppl:
            
            first_name = name.text.replace("\n"," ").strip().split()[0]
            #print(name.text.replace("\n"," ").strip())
            
            if first_name not in known_names:
                #print(first_name,'not recognized!')
                continue

            x = genderize.loc[first_name]
            #print(first_name,x["gender"],x['probability'])
            genders.append( 1.0 * (x["gender"] == 'female') )
            genderprobs.append(x['probability'])
        if genders: item['Is Female (Weighted)'] = np.average(genders,weights=genderprobs)

        sentlist.append(item)
        
    return sentlist

In [129]:
dx=[]
for f in F_CSV:
    df = pd.read_csv(f)#, nrows=15000)
    
    #dfunc = joblib.delayed(parse_df)
    #with joblib.Parallel(1) as MP:
    #    dx = MP(dfunc(row,k) for k,row in df.iterrows())
    
    for k,row in df.iterrows():
        dx.extend(parse_df(row,k))
     
    dxtmp = pd.DataFrame(dx).set_index('filename')
    f_save = os.path.join("../parsed_data/sentence_parse/",
                          'gendered_sentences.csv.part')
    dxtmp.to_csv(f_save)
    
dx = pd.DataFrame(dx).set_index('filename')
f_save = os.path.join("../parsed_data/sentence_parse/",
                          'gendered_sentences.csv')
dx.to_csv(f_save)
dx

0
Tue Sep 11 21:13:09 2018
1000
Tue Sep 11 21:13:35 2018
2000
Tue Sep 11 21:14:05 2018
3000
Tue Sep 11 21:14:51 2018
4000
Tue Sep 11 21:15:39 2018
5000
Tue Sep 11 21:16:18 2018
6000
Tue Sep 11 21:17:05 2018
7000
Tue Sep 11 21:18:23 2018
8000
Tue Sep 11 21:18:40 2018
9000
Tue Sep 11 21:19:05 2018
10000
Tue Sep 11 21:19:49 2018
11000
Tue Sep 11 21:20:12 2018
12000
Tue Sep 11 21:20:32 2018
13000
Tue Sep 11 21:20:54 2018
14000
Tue Sep 11 21:21:13 2018
15000
Tue Sep 11 21:21:31 2018
16000
Tue Sep 11 21:21:49 2018
17000
Tue Sep 11 21:22:08 2018
18000
Tue Sep 11 21:22:28 2018
19000
Tue Sep 11 21:22:47 2018
20000
Tue Sep 11 21:23:07 2018
21000
Tue Sep 11 21:23:27 2018
22000
Tue Sep 11 21:23:48 2018
23000
Tue Sep 11 21:24:08 2018
24000
Tue Sep 11 21:24:28 2018
25000
Tue Sep 11 21:24:48 2018
26000
Tue Sep 11 21:25:09 2018
27000
Tue Sep 11 21:25:29 2018
28000
Tue Sep 11 21:25:50 2018
29000
Tue Sep 11 21:26:12 2018
30000
Tue Sep 11 21:26:34 2018
31000
Tue Sep 11 21:26:57 2018
32000
Tue Sep 11 21:2

260000
Tue Sep 11 23:57:03 2018
261000
Tue Sep 11 23:58:10 2018
262000
Tue Sep 11 23:58:50 2018
263000
Tue Sep 11 23:59:01 2018
264000
Tue Sep 11 23:59:54 2018
265000
Wed Sep 12 00:00:32 2018
266000
Wed Sep 12 00:00:52 2018
267000
Wed Sep 12 00:02:47 2018
268000
Wed Sep 12 00:03:38 2018
269000
Wed Sep 12 00:03:38 2018
270000
Wed Sep 12 00:03:39 2018
271000
Wed Sep 12 00:04:02 2018
272000
Wed Sep 12 00:04:31 2018
273000
Wed Sep 12 00:04:57 2018
274000
Wed Sep 12 00:05:24 2018
275000
Wed Sep 12 00:05:52 2018
276000
Wed Sep 12 00:06:21 2018
277000
Wed Sep 12 00:06:47 2018
278000
Wed Sep 12 00:07:14 2018
279000
Wed Sep 12 00:07:44 2018
280000
Wed Sep 12 00:08:10 2018
281000
Wed Sep 12 00:08:30 2018
282000
Wed Sep 12 00:09:00 2018
283000
Wed Sep 12 00:09:25 2018
284000
Wed Sep 12 00:09:55 2018
285000
Wed Sep 12 00:10:19 2018
286000
Wed Sep 12 00:11:05 2018
287000
Wed Sep 12 00:11:31 2018
288000
Wed Sep 12 00:12:01 2018
289000
Wed Sep 12 00:12:35 2018
290000
Wed Sep 12 00:13:03 2018
291000
W

141000
Wed Sep 12 01:40:56 2018
142000
Wed Sep 12 01:42:01 2018
143000
Wed Sep 12 01:42:39 2018
144000
Wed Sep 12 01:42:48 2018
145000
Wed Sep 12 01:42:55 2018
146000
Wed Sep 12 01:43:03 2018
147000
Wed Sep 12 01:43:20 2018
148000
Wed Sep 12 01:43:36 2018
149000
Wed Sep 12 01:43:53 2018
150000
Wed Sep 12 01:44:11 2018
151000
Wed Sep 12 01:44:39 2018
152000
Wed Sep 12 01:45:12 2018
153000
Wed Sep 12 01:45:21 2018
154000
Wed Sep 12 01:45:47 2018
155000
Wed Sep 12 01:45:56 2018
156000
Wed Sep 12 01:46:37 2018
157000
Wed Sep 12 01:47:31 2018
158000
Wed Sep 12 01:47:52 2018
159000
Wed Sep 12 01:48:15 2018
160000
Wed Sep 12 01:48:43 2018
161000
Wed Sep 12 01:49:08 2018
162000
Wed Sep 12 01:49:28 2018
163000
Wed Sep 12 01:49:59 2018
164000
Wed Sep 12 01:50:33 2018
165000
Wed Sep 12 01:50:55 2018
166000
Wed Sep 12 01:51:20 2018
167000
Wed Sep 12 01:51:52 2018
168000
Wed Sep 12 01:52:31 2018
169000
Wed Sep 12 01:52:52 2018
170000
Wed Sep 12 01:53:16 2018
171000
Wed Sep 12 01:53:38 2018
172000
W

398000
Wed Sep 12 03:28:46 2018
399000
Wed Sep 12 03:29:15 2018
400000
Wed Sep 12 03:29:27 2018
401000
Wed Sep 12 03:29:55 2018
402000
Wed Sep 12 03:30:21 2018
403000
Wed Sep 12 03:30:36 2018
404000
Wed Sep 12 03:30:59 2018
405000
Wed Sep 12 03:31:45 2018
406000
Wed Sep 12 03:31:55 2018
407000
Wed Sep 12 03:32:02 2018
408000
Wed Sep 12 03:32:11 2018
409000
Wed Sep 12 03:32:41 2018
410000
Wed Sep 12 03:32:58 2018
411000
Wed Sep 12 03:33:18 2018
412000
Wed Sep 12 03:33:34 2018
413000
Wed Sep 12 03:34:08 2018
414000
Wed Sep 12 03:34:33 2018
415000
Wed Sep 12 03:34:56 2018
416000
Wed Sep 12 03:35:13 2018
417000
Wed Sep 12 03:35:33 2018
418000
Wed Sep 12 03:36:02 2018
419000
Wed Sep 12 03:36:28 2018
420000
Wed Sep 12 03:36:48 2018
421000
Wed Sep 12 03:36:52 2018
422000
Wed Sep 12 03:37:24 2018
423000
Wed Sep 12 03:37:47 2018
424000
Wed Sep 12 03:38:14 2018
425000
Wed Sep 12 03:38:40 2018
426000
Wed Sep 12 03:38:43 2018
427000
Wed Sep 12 03:39:28 2018
428000
Wed Sep 12 03:41:55 2018
429000
W

1000
Wed Sep 12 05:46:28 2018
2000
Wed Sep 12 05:46:53 2018
3000
Wed Sep 12 05:47:24 2018
4000
Wed Sep 12 05:47:36 2018
5000
Wed Sep 12 05:47:54 2018
6000
Wed Sep 12 05:48:16 2018
7000
Wed Sep 12 05:49:03 2018
8000
Wed Sep 12 05:50:03 2018
9000
Wed Sep 12 05:50:28 2018
10000
Wed Sep 12 05:50:40 2018
11000
Wed Sep 12 05:50:55 2018
12000
Wed Sep 12 05:51:13 2018
13000
Wed Sep 12 05:51:34 2018
14000
Wed Sep 12 05:51:55 2018
15000
Wed Sep 12 05:52:19 2018
16000
Wed Sep 12 05:52:42 2018
17000
Wed Sep 12 05:53:17 2018
18000
Wed Sep 12 05:53:55 2018
19000
Wed Sep 12 05:54:33 2018
20000
Wed Sep 12 05:55:05 2018
21000
Wed Sep 12 05:55:40 2018
22000
Wed Sep 12 05:56:12 2018
23000
Wed Sep 12 05:56:45 2018
24000
Wed Sep 12 05:57:16 2018
25000
Wed Sep 12 05:57:48 2018
26000
Wed Sep 12 05:58:20 2018
27000
Wed Sep 12 05:58:50 2018
28000
Wed Sep 12 05:59:19 2018
29000
Wed Sep 12 05:59:46 2018
30000
Wed Sep 12 06:00:13 2018
31000
Wed Sep 12 06:00:39 2018
32000
Wed Sep 12 06:01:04 2018
33000
Wed Sep 12 

261000
Wed Sep 12 08:16:31 2018
262000
Wed Sep 12 08:17:08 2018
263000
Wed Sep 12 08:17:45 2018
264000
Wed Sep 12 08:18:23 2018
265000
Wed Sep 12 08:19:04 2018
266000
Wed Sep 12 08:19:42 2018
267000
Wed Sep 12 08:20:14 2018
268000
Wed Sep 12 08:20:50 2018
269000
Wed Sep 12 08:21:26 2018
270000
Wed Sep 12 08:22:09 2018
271000
Wed Sep 12 08:22:48 2018
272000
Wed Sep 12 08:23:26 2018
273000
Wed Sep 12 08:24:08 2018
274000
Wed Sep 12 08:24:47 2018
275000
Wed Sep 12 08:25:26 2018
276000
Wed Sep 12 08:26:03 2018
277000
Wed Sep 12 08:26:43 2018
278000
Wed Sep 12 08:27:20 2018
279000
Wed Sep 12 08:28:02 2018
280000
Wed Sep 12 08:28:45 2018
281000
Wed Sep 12 08:29:24 2018
282000
Wed Sep 12 08:30:03 2018
283000
Wed Sep 12 08:30:38 2018
284000
Wed Sep 12 08:31:17 2018
285000
Wed Sep 12 08:31:55 2018
286000
Wed Sep 12 08:32:31 2018
287000
Wed Sep 12 08:33:09 2018
288000
Wed Sep 12 08:33:49 2018
289000
Wed Sep 12 08:34:27 2018
290000
Wed Sep 12 08:35:01 2018
291000
Wed Sep 12 08:35:42 2018
292000
W

518000
Wed Sep 12 10:56:45 2018
519000
Wed Sep 12 10:57:55 2018
520000
Wed Sep 12 10:58:53 2018
521000
Wed Sep 12 10:59:27 2018
522000
Wed Sep 12 11:00:59 2018
523000
Wed Sep 12 11:01:26 2018
524000
Wed Sep 12 11:01:59 2018
525000
Wed Sep 12 11:02:23 2018
526000
Wed Sep 12 11:03:08 2018
527000
Wed Sep 12 11:03:53 2018
528000
Wed Sep 12 11:04:43 2018
529000
Wed Sep 12 11:04:48 2018
530000
Wed Sep 12 11:04:53 2018
531000
Wed Sep 12 11:05:19 2018
532000
Wed Sep 12 11:06:27 2018
533000
Wed Sep 12 11:06:49 2018
534000
Wed Sep 12 11:07:21 2018
535000
Wed Sep 12 11:07:32 2018
536000
Wed Sep 12 11:07:51 2018
537000
Wed Sep 12 11:08:45 2018
538000
Wed Sep 12 11:09:08 2018
539000
Wed Sep 12 11:09:33 2018
540000
Wed Sep 12 11:10:04 2018
541000
Wed Sep 12 11:10:28 2018
542000
Wed Sep 12 11:10:55 2018
543000
Wed Sep 12 11:11:28 2018
544000
Wed Sep 12 11:11:59 2018
545000
Wed Sep 12 11:11:59 2018
546000
Wed Sep 12 11:11:59 2018
547000
Wed Sep 12 11:11:59 2018
548000
Wed Sep 12 11:12:30 2018
549000
W

167000
Wed Sep 12 13:05:38 2018
168000
Wed Sep 12 13:06:21 2018
169000
Wed Sep 12 13:07:34 2018
170000
Wed Sep 12 13:08:33 2018
171000
Wed Sep 12 13:08:53 2018
172000
Wed Sep 12 13:09:01 2018
173000
Wed Sep 12 13:09:16 2018
174000
Wed Sep 12 13:10:07 2018
175000
Wed Sep 12 13:11:09 2018
176000
Wed Sep 12 13:12:13 2018
177000
Wed Sep 12 13:13:17 2018
178000
Wed Sep 12 13:14:03 2018
179000
Wed Sep 12 13:14:03 2018
180000
Wed Sep 12 13:14:03 2018
181000
Wed Sep 12 13:14:03 2018
182000
Wed Sep 12 13:14:03 2018
183000
Wed Sep 12 13:14:03 2018
184000
Wed Sep 12 13:14:03 2018
185000
Wed Sep 12 13:14:03 2018
186000
Wed Sep 12 13:14:03 2018
187000
Wed Sep 12 13:14:04 2018
188000
Wed Sep 12 13:14:04 2018
189000
Wed Sep 12 13:14:04 2018
190000
Wed Sep 12 13:14:04 2018
191000
Wed Sep 12 13:14:04 2018
192000
Wed Sep 12 13:14:04 2018
193000
Wed Sep 12 13:14:04 2018
194000
Wed Sep 12 13:14:04 2018
195000
Wed Sep 12 13:14:04 2018
196000
Wed Sep 12 13:14:04 2018
197000
Wed Sep 12 13:14:04 2018
198000
W

428000
Wed Sep 12 15:05:37 2018
429000
Wed Sep 12 15:05:37 2018
430000
Wed Sep 12 15:05:37 2018
431000
Wed Sep 12 15:05:37 2018
432000
Wed Sep 12 15:05:37 2018
433000
Wed Sep 12 15:05:37 2018
434000
Wed Sep 12 15:05:37 2018
435000
Wed Sep 12 15:05:37 2018
436000
Wed Sep 12 15:05:37 2018
437000
Wed Sep 12 15:05:37 2018
438000
Wed Sep 12 15:05:38 2018
439000
Wed Sep 12 15:05:38 2018
440000
Wed Sep 12 15:05:38 2018
441000
Wed Sep 12 15:05:38 2018
442000
Wed Sep 12 15:05:38 2018
443000
Wed Sep 12 15:05:38 2018
444000
Wed Sep 12 15:05:38 2018
445000
Wed Sep 12 15:05:38 2018
446000
Wed Sep 12 15:05:38 2018
447000
Wed Sep 12 15:05:38 2018
448000
Wed Sep 12 15:05:38 2018
449000
Wed Sep 12 15:05:38 2018
450000
Wed Sep 12 15:05:39 2018
451000
Wed Sep 12 15:05:39 2018
452000
Wed Sep 12 15:05:39 2018
453000
Wed Sep 12 15:05:39 2018
454000
Wed Sep 12 15:06:15 2018
455000
Wed Sep 12 15:07:01 2018
456000
Wed Sep 12 15:08:17 2018
457000
Wed Sep 12 15:09:10 2018
458000
Wed Sep 12 15:09:30 2018
459000
W

,Is Female (Weighted),Names,Nouns,Organizations,Text,Verbs
filename,,,,,,
20_Century_Br_Hist/PMC4804230.nxml,0.000000,Mathew Thomson,author;reviewers;comments;drafts;article,,The author would like to sincerely thank the a...,like;thank
20_Century_Br_Hist/PMC5905639.nxml,0.000000,Mathew Thomson,co;investigator;members;team;comments;insight,,I would like to thank my co-investigator Mathe...,like;thank
3D_Print_Med/PMC5954789.nxml,NaN,Diya Ma;Matthew-Lun Wong;Jin-Peng Lee,authors;contributions;software;development,,"The authors wish to acknowledge Diya Ma, Matth...",wish;acknowledge
3D_Print_Med/PMC5954790.nxml,1.000000,Clarice Smith;Abigail van Buren;Corinne Schuler,support,the Robert Wood Johnson Foundation;Marvin Deke...,Additional support was provided by the Robert ...,provide
3D_Print_Med/PMC5954792.nxml,0.000000,Michel Vong,authors;help;operation;printing;video;recordin...,the University of Edinburgh,The authors would like to thank Michel Vong of...,like;thank;support
3D_Print_Med/PMC5954794.nxml,NaN,Dinghau Huang;Pi-Hsia Liu,authors;arrangement;participants;orthosis;desi...,Ming Chi University of Technology;the Chang Gu...,The authors would like to thank Dr. Dinghau Hu...,like;thank;concern
3D_Print_Med/PMC5954795.nxml,0.000000,Alexandre Colello Bruno;Agnelo Bastos;Clésio F...,contribution;support,,Alexandre Colello Bruno for intellectual contr...,
3D_Print_Med/PMC5954799.nxml,0.000000,Dustin Makin,authors;support;3D;process,CreateCafe;Saskatoon,"The authors would like to thank Dustin Makin, ...",like;thank;print
3D_Print_Med/PMC6036666.nxml,0.000000,Thomas Vrieze;Kris Nunez,authors;assistance;data;acquisition;assistance...,,The authors would like to thank Thomas Vrieze ...,like;thank
